In [1]:
#!pip install python-dotenv decorator==4.3 networkx
#!pip install pycoingecko
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import pprint


%matplotlib inline

In [2]:
load_dotenv('../.env')

True

In [3]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [4]:
coin_market = cg.get_coins_markets(vs_currency='usd')
coin_market_df= pd.DataFrame(coin_market, columns=['id', 'current_price', 'high_24h' , 'low_24h'])
coin_market_df.set_index('id', inplace=True)
coin_market_df

,current_price,high_24h,low_24h
id,,,
bitcoin,51686.000000,51971.000000,49231.000000
ethereum,3501.470000,3540.820000,3377.660000
cardano,2.220000,2.260000,2.190000
tether,1.000000,1.010000,0.996900
binancecoin,438.570000,444.620000,426.380000
...,...,...,...
yearn-finance,31570.000000,32257.000000,30206.000000
telcoin,0.019271,0.019882,0.018515
basic-attention-token,0.739309,0.745446,0.704706


In [5]:
btc_history = cg.get_coin_market_chart_by_id(id= 'bitcoin', vs_currency = 'usd', days = 'max')
btc_history_df = pd.DataFrame(btc_history)
btc_history_df

,prices,market_caps,total_volumes
0,"[1367107200000, 135.3]","[1367107200000, 1500517590]","[1367107200000, 0]"
1,"[1367193600000, 141.96]","[1367193600000, 1575032004.0]","[1367193600000, 0.0]"
2,"[1367280000000, 135.3]","[1367280000000, 1501657493.0]","[1367280000000, 0.0]"
3,"[1367366400000, 117.0]","[1367366400000, 1298951550.0]","[1367366400000, 0.0]"
4,"[1367452800000, 103.43]","[1367452800000, 1148667722.0]","[1367452800000, 0.0]"
...,...,...,...
3076,"[1633046400000, 43859.32614724109]","[1633046400000, 827840965059.6127]","[1633046400000, 30859918334.565014]"
3077,"[1633132800000, 48181.87627260008]","[1633132800000, 906929641888.8245]","[1633132800000, 42089383189.81192]"
3078,"[1633219200000, 47777.762019063]","[1633219200000, 900582483713.1118]","[1633219200000, 29742127387.956066]"
3079,"[1633305600000, 48282.97113857447]","[1633305600000, 910331395887.4816]","[1633305600000, 26034594561.963406]"


In [6]:
eth_history = cg.get_coin_market_chart_by_id(id= 'ethereum', vs_currency = 'usd', days = 'max')
eth_history_df = pd.DataFrame(eth_history)
eth_history_df

,prices,market_caps,total_volumes
0,"[1438905600000, 2.83162]","[1438905600000, 0.0]","[1438905600000, 90622.0]"
1,"[1438992000000, 1.33075]","[1438992000000, 80339475.0]","[1438992000000, 368070.0]"
2,"[1439164800000, 0.6875857090838133]","[1439164800000, 41556309.21112176]","[1439164800000, 400464.0745435489]"
3,"[1439251200000, 1.067378879429946]","[1439251200000, 64539006.306272194]","[1439251200000, 1518998.2053373822]"
4,"[1439337600000, 1.256613175101658]","[1439337600000, 76013261.05415402]","[1439337600000, 2073893.3450314845]"
...,...,...,...
2247,"[1633132800000, 3305.107041443724]","[1633132800000, 388950236002.1381]","[1633132800000, 24368105704.246353]"
2248,"[1633219200000, 3393.9242492696653]","[1633219200000, 399455864987.66345]","[1633219200000, 21335051735.331825]"
2249,"[1633305600000, 3426.387080851377]","[1633305600000, 403971441903.55585]","[1633305600000, 18130363405.685154]"
2250,"[1633392000000, 3390.310407759615]","[1633392000000, 400955543856.1877]","[1633392000000, 19898468513.475533]"


In [7]:
lte_history = cg.get_coin_market_chart_by_id(id= 'litecoin', vs_currency = 'usd', days = 'max')
lte_history_df = pd.DataFrame(lte_history)
lte_history_df

,prices,market_caps,total_volumes
0,"[1367107200000, 4.29983]","[1367107200000, 73773387]","[1367107200000, 0]"
1,"[1367193600000, 4.3594]","[1367193600000, 74936909.0]","[1367193600000, 0.0]"
2,"[1367280000000, 4.18295]","[1367280000000, 72037636.0]","[1367280000000, 0.0]"
3,"[1367366400000, 3.64914]","[1367366400000, 62957992.0]","[1367366400000, 0.0]"
4,"[1367452800000, 3.38879]","[1367452800000, 58565340.0]","[1367452800000, 0.0]"
...,...,...,...
3077,"[1633132800000, 166.03298444923087]","[1633132800000, 11077015406.904276]","[1633132800000, 2943230546.085889]"
3078,"[1633219200000, 169.65830733575754]","[1633219200000, 11659851024.782589]","[1633219200000, 2859712957.102274]"
3079,"[1633305600000, 170.96527037444142]","[1633305600000, 11749810607.616468]","[1633305600000, 2912326781.574752]"
3080,"[1633392000000, 167.54549258443683]","[1633392000000, 11506766114.666618]","[1633392000000, 2685181954.2209654]"


In [8]:
link_history = cg.get_coin_market_chart_by_id(id= 'chainlink', vs_currency = 'usd', days = 'max')
link_history_df = pd.DataFrame(link_history)b
link_history_df

,prices,market_caps,total_volumes
0,"[1510185600000, 0.22537665242356383]","[1510185600000, 78881828.34824735]","[1510185600000, 3190025.855165697]"
1,"[1510272000000, 0.189036422857878]","[1510272000000, 66162748.000257306]","[1510272000000, 2121175.2731304658]"
2,"[1510358400000, 0.18122267085020952]","[1510358400000, 63427934.797573335]","[1510358400000, 2229297.6966014686]"
3,"[1510444800000, 0.1700132017426718]","[1510444800000, 59504620.60993513]","[1510444800000, 5361145.800394995]"
4,"[1510531200000, 0.1806910270317246]","[1510531200000, 63241859.46110361]","[1510531200000, 2942234.064639167]"
...,...,...,...
1422,"[1633046400000, 23.92999747979145]","[1633046400000, 10859891607.807625]","[1633046400000, 921098524.958979]"
1423,"[1633132800000, 26.310431364502723]","[1633132800000, 11985438391.491632]","[1633132800000, 1209726617.1253884]"
1424,"[1633219200000, 26.974247784337233]","[1633219200000, 12319787469.834383]","[1633219200000, 975093495.5204914]"
1425,"[1633305600000, 27.609012037186666]","[1633305600000, 12627009283.328642]","[1633305600000, 914842196.7789143]"


In [9]:
ada_history = cg.get_coin_market_chart_by_id(id= 'cardano', vs_currency = 'usd', days = 'max')
ada_history_df = pd.DataFrame(ada_history)
ada_history_df

,prices,market_caps,total_volumes
0,"[1508284800000, 0.02684535467621909]","[1508284800000, 696021404.3079604]","[1508284800000, 2351678.122306208]"
1,"[1508371200000, 0.026829950465728686]","[1508371200000, 695622018.2559936]","[1508371200000, 2815155.9695999967]"
2,"[1508457600000, 0.030299606216252807]","[1508457600000, 785580027.6424098]","[1508457600000, 8883473.082062634]"
3,"[1508544000000, 0.028587962639027336]","[1508544000000, 741202123.8797704]","[1508544000000, 5308856.917095766]"
4,"[1508630400000, 0.027796038407662104]","[1508630400000, 720669848.4724126]","[1508630400000, 2901876.1220860113]"
...,...,...,...
1444,"[1633046400000, 2.1166519301521833]","[1633046400000, 67871265795.13519]","[1633046400000, 2092736072.8648612]"
1445,"[1633132800000, 2.2467692592328996]","[1633132800000, 71980741780.63368]","[1633132800000, 2511823046.243126]"
1446,"[1633219200000, 2.2472190210882728]","[1633219200000, 72017933348.18193]","[1633219200000, 2194904017.547218]"
1447,"[1633305600000, 2.2588385813642846]","[1633305600000, 72529381282.70782]","[1633305600000, 1560814022.0556097]"


In [10]:
#Coin Merged DataFrame
#merged_coin_df = pd.concat([btc_history_df, eth_history_df])

In [11]:
#Alpaca Keys
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [12]:
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [13]:
aapl_data = api.get_barset('AAPL',timeframe='1D', limit=100,).df
aapl_data.head()

AAPL                                   
                             open     high     low   close    volume
time                                                                
2021-05-14 00:00:00-04:00  126.27  127.890  125.85  127.44  68037466
2021-05-17 00:00:00-04:00  126.82  126.930  125.17  126.26  59901221
2021-05-18 00:00:00-04:00  126.56  126.990  124.78  124.85  53519167
2021-05-19 00:00:00-04:00  123.16  124.915  122.86  124.69  76587655
2021-05-20 00:00:00-04:00  125.23  127.720  125.10  127.31  65578504

In [14]:
amzn_data = api.get_barset('AMZN',timeframe='1D', limit=100,).df
amzn_data.head()

AMZN                                    
                              open     high      low    close   volume
time                                                                  
2021-05-14 00:00:00-04:00  3185.00  3228.86  3183.00  3221.92  2660882
2021-05-17 00:00:00-04:00  3246.25  3292.75  3234.59  3271.79  3231546
2021-05-18 00:00:00-04:00  3293.17  3312.00  3230.37  3232.73  2500745
2021-05-19 00:00:00-04:00  3196.80  3234.75  3184.00  3232.55  2293730
2021-05-20 00:00:00-04:00  3247.59  3259.68  3236.18  3248.54  2183348

In [15]:
ba_data = api.get_barset('BA',timeframe='1D', limit=100,).df
ba_data.head()

BA                                     
                             open      high     low    close    volume
time                                                                  
2021-05-14 00:00:00-04:00  225.78  228.8100  224.35  228.488   9200121
2021-05-17 00:00:00-04:00  226.33  228.2600  223.58  227.990   9151534
2021-05-18 00:00:00-04:00  227.69  232.2652  226.43  227.550   9622918
2021-05-19 00:00:00-04:00  223.40  225.1200  220.28  224.375  12563064
2021-05-20 00:00:00-04:00  225.10  228.4389  223.77  227.710   8767011

In [16]:
nvda_data = api.get_barset('NVDA',timeframe='1D', limit=100,).df
nvda_data.head()

NVDA                                     
                             open      high     low     close   volume
time                                                                  
2021-05-14 00:00:00-04:00  555.00  573.1242  549.94  569.6600  5665040
2021-05-17 00:00:00-04:00  566.43  566.7700  554.14  566.7700  4818845
2021-05-18 00:00:00-04:00  570.97  576.1700  560.16  560.5725  4243935
2021-05-19 00:00:00-04:00  542.85  563.1800  541.73  562.9700  7628866
2021-05-20 00:00:00-04:00  572.10  587.3514  570.82  584.5000  7312140

In [17]:
brk_data = api.get_barset('BRK.A',timeframe='1D', limit=100,).df
brk_data.head()

BRK.A                                      
                                open      high       low      close volume
time                                                                      
2021-05-14 00:00:00-04:00  433995.50  437413.0  433995.5  437259.97   2076
2021-05-17 00:00:00-04:00  435914.80  436131.0  432747.0  434324.00   1651
2021-05-18 00:00:00-04:00  435440.00  435960.0  431767.0  431767.00   1261
2021-05-19 00:00:00-04:00  426930.56  430570.0  424586.0  430126.50   1676
2021-05-20 00:00:00-04:00  432773.06  435000.0  429879.5  431069.50   1601

In [18]:
gs_data = api.get_barset('GS',timeframe='1D', limit=100,).df
gs_data.head()

GS                                   
                             open    high       low   close   volume
time                                                                
2021-05-14 00:00:00-04:00  362.00  369.51  361.7000  368.76  2352615
2021-05-17 00:00:00-04:00  368.33  370.70  364.5000  369.34  1463950
2021-05-18 00:00:00-04:00  371.38  373.08  364.0000  364.42  1587128
2021-05-19 00:00:00-04:00  358.75  360.00  353.9319  358.39  2619338
2021-05-20 00:00:00-04:00  358.79  361.95  353.3500  359.88  1705627

In [19]:
cop_data = api.get_barset('COP',timeframe='1D', limit=100,).df
cop_data.head()

COP                                
                            open    high     low  close   volume
time                                                            
2021-05-14 00:00:00-04:00  55.03  56.645  55.030  56.52  5223778
2021-05-17 00:00:00-04:00  56.07  58.310  56.070  58.31  8796264
2021-05-18 00:00:00-04:00  58.27  58.640  56.960  57.29  9355731
2021-05-19 00:00:00-04:00  56.13  56.530  54.815  55.59  8242435
2021-05-20 00:00:00-04:00  55.65  55.660  54.634  55.38  7857981

In [20]:
gild_data = api.get_barset('GILD',timeframe='1D', limit=100,).df
gild_data.head()

GILD                                
                            open   high      low  close   volume
time                                                            
2021-05-14 00:00:00-04:00  68.90  69.18  68.3884  68.53  6039858
2021-05-17 00:00:00-04:00  68.46  69.65  68.4500  69.08  5210995
2021-05-18 00:00:00-04:00  69.41  69.87  68.8000  68.85  5017773
2021-05-19 00:00:00-04:00  68.27  68.57  67.5500  68.50  4690703
2021-05-20 00:00:00-04:00  68.66  69.98  68.3700  69.35  5435990

In [21]:
ccoi_data = api.get_barset('CCOI',timeframe='1D', limit=100,).df
ccoi_data.head()

CCOI                               
                            open     high    low  close  volume
time                                                           
2021-05-14 00:00:00-04:00  77.31  77.6381  76.30  77.21  106270
2021-05-17 00:00:00-04:00  76.71  76.7100  75.25  75.37  210295
2021-05-18 00:00:00-04:00  75.41  75.9555  74.68  74.79  256984
2021-05-19 00:00:00-04:00  74.29  75.2700  73.56  74.19  351061
2021-05-20 00:00:00-04:00  73.90  75.1200  73.90  74.57  368751

In [22]:
proc_data = api.get_barset('PROC',timeframe='1D', limit=100,).df
proc_data.head()

PROC                             
                            open    high   low  close  volume
time                                                         
2021-09-30 00:00:00-04:00   8.41  10.500  8.41  10.00  581758
2021-10-01 00:00:00-04:00  10.38  10.400  9.50   9.59  802142
2021-10-04 00:00:00-04:00   9.80   9.925  8.13   9.60  442088
2021-10-05 00:00:00-04:00   9.46   9.800  8.90   9.79   92723

In [23]:
# aapl_data.dropna(inplace=True)
# amzn_data.dropna(inplace=True)
# ba_data.dropna(inplace=True)
# nvda_data.dropna(inplace=True)
# brk_data.dropna(inplace=True)
# gs_data.dropna(inplace=True)
# cop_data.dropna(inplace=True) 
# gild_data.dropna(inplace=True)
# ccoi_data.dropna(inplace=True)
# proc_data.dropna(inplace=True)

In [24]:
merged_stock_df= pd.concat([
    aapl_data, amzn_data, ba_data, nvda_data, brk_data,
    gs_data, cop_data, gild_data, ccoi_data, proc_data], axis='columns', join='inner')
#merged_stock_df = pd.concat(to_merge_dict.values())
#merged_stock_df.dropna(inplace=True)
merged_stock_df.head()

AAPL                                       \
                             open     high       low   close    volume   
time                                                                     
2021-09-30 00:00:00-04:00  143.66  144.378  141.2800  141.32  76345831   
2021-10-01 00:00:00-04:00  141.83  142.920  139.1101  142.66  81585491   
2021-10-04 00:00:00-04:00  141.71  142.210  138.2700  139.19  86291346   
2021-10-05 00:00:00-04:00  139.49  142.240  139.3600  141.10  68998178   

                              AMZN                                         \
                              open       high       low    close   volume   
time                                                                        
2021-09-30 00:00:00-04:00  3315.25  3327.8500  3273.990  3286.61  2349154   
2021-10-01 00:00:00-04:00  3289.01  3309.1746  3255.940  3283.17  2513022   
2021-10-04 00:00:00-04:00  3278.75  3279.9900  3176.250  3191.70  4034895   
2021-10-05 00:00:00-04:00  3204.12  3260.7300  3202.464  3220.99  2672848   

                           ...   CCOI                                PROC  \
                           ...   open   high    low  close  volume   open   
time                       ...                                              
2021-09-30 00:00:00-04:00  ...  71.50  71.79  70.49  70.75  117708   8.41   
2021-10-01 00:00:00-04:00  ...  71.21  72.36  69.82  72.16  150384  10.38   
2021-10-04 00:00:00-04:00  ...  72.26  73.07  71.65  72.98  112374   9.80   
2021-10-05 00:00:00-04:00  ...  72.94  72.94  71.95  72.41   97653   9.46   

                                                        
                             high   low  close  volume  
time                                                    
2021-09-30 00:00:00-04:00  10.500  8.41  10.00  581758  
2021-10-01 00:00:00-04:00  10.400  9.50   9.59  802142  
2021-10-04 00:00:00-04:00   9.925  8.13   9.60  442088  
2021-10-05 00:00:00-04:00   9.800  8.90   9.79   92723  

[4 rows x 50 columns]

In [25]:
#Bond Data
#iSHares 20 year plus treasury
tlt_data = api.get_barset('TLT',timeframe='1D', limit=1000,).df
tlt_data.head()

TLT                                 
                             open    high     low   close   volume
time                                                              
2017-10-16 00:00:00-04:00  125.53  125.90  125.38  125.75  4944568
2017-10-17 00:00:00-04:00  125.39  126.09  125.39  125.92  3961713
2017-10-18 00:00:00-04:00  125.05  125.18  124.68  125.10  4706805
2017-10-19 00:00:00-04:00  125.74  126.01  125.29  125.29  7882061
2017-10-20 00:00:00-04:00  124.00  124.28  123.64  124.01  9720777

In [26]:
#Longterm treasury shares
vglt_data = api.get_barset('VGLT',timeframe='1D', limit=1000,).df
vglt_data.head()

VGLT                                
                            open    high      low  close  volume
time                                                            
2017-10-11 00:00:00-04:00  76.74  76.830  76.5800  76.68   66600
2017-10-12 00:00:00-04:00  76.73  77.020  76.5527  77.00   23922
2017-10-13 00:00:00-04:00  77.34  77.570  77.1400  77.52   75208
2017-10-16 00:00:00-04:00  77.31  77.525  77.2500  77.40   76080
2017-10-17 00:00:00-04:00  77.24  77.620  77.2400  77.57  258788

In [27]:
#Vanguard S&P 500 Growth
voog_data = api.get_barset('VOOG',timeframe='1D', limit=1000,).df
voog_data.head()

VOOG                                     
                             open      high       low     close volume
time                                                                  
2017-10-12 00:00:00-04:00  130.72  131.1336  130.7000  130.8401  33853
2017-10-13 00:00:00-04:00  131.26  131.3690  131.1000  131.1100  27520
2017-10-16 00:00:00-04:00  131.43  131.4596  131.0150  131.3600  31031
2017-10-17 00:00:00-04:00  131.27  131.5900  131.2600  131.5800  29113
2017-10-18 00:00:00-04:00  131.88  131.8800  131.5301  131.6299  39530

In [28]:
#Vanguard Mortgage Backed Securities
vmbs_data = api.get_barset('VMBS',timeframe='1D', limit=1000,).df
vmbs_data.head()

VMBS                               
                            open     high    low  close  volume
time                                                           
2017-10-11 00:00:00-04:00  52.77  52.8100  52.72  52.74  916920
2017-10-12 00:00:00-04:00  52.81  52.8300  52.76  52.80  275237
2017-10-13 00:00:00-04:00  52.89  52.9000  52.47  52.87  239158
2017-10-16 00:00:00-04:00  52.86  52.8800  52.47  52.83  251857
2017-10-17 00:00:00-04:00  52.84  52.8499  52.80  52.82  441153

In [29]:
#iShares Russell 1000 Growth ETF
iwf_data = api.get_barset('IWF',timeframe='1D', limit=1000,).df
iwf_data.head()

IWF                                   
                             open      high     low   close   volume
time                                                                
2017-10-12 00:00:00-04:00  127.11  127.6800  127.11  127.42   590195
2017-10-13 00:00:00-04:00  127.79  127.9400  127.61  127.68   789672
2017-10-16 00:00:00-04:00  127.81  127.9800  127.57  127.88   775453
2017-10-17 00:00:00-04:00  127.80  128.0600  127.80  128.01  1547576
2017-10-18 00:00:00-04:00  128.33  128.3353  128.00  128.14   862874

In [30]:
#iShares 1-5 year Investment grade bond
igsb_data = api.get_barset('IGSB',timeframe='1D', limit=1000,).df
igsb_data.head()

IGSB                                
                             open    high     low   close  volume
time                                                             
2017-10-13 00:00:00-04:00  52.620  52.675  52.615  52.645  916382
2017-10-16 00:00:00-04:00  52.610  52.625  52.590  52.593  550776
2017-10-17 00:00:00-04:00  52.590  52.611  52.580  52.599  621174
2017-10-18 00:00:00-04:00  52.575  52.615  52.569  52.615  446712
2017-10-19 00:00:00-04:00  52.635  52.645  52.605  52.630  529644

In [31]:
#iShares US Treasury Bond ETF
govt_data = api.get_barset('GOVT',timeframe='1D', limit=1000,).df
govt_data.head()

GOVT                               
                            open    high    low  close   volume
time                                                           
2017-10-16 00:00:00-04:00  25.28  25.280  25.23  25.26  1713890
2017-10-17 00:00:00-04:00  25.23  25.260  25.21  25.25   675884
2017-10-18 00:00:00-04:00  25.20  25.210  25.17  25.20   507468
2017-10-19 00:00:00-04:00  25.25  25.265  25.22  25.23   689206
2017-10-20 00:00:00-04:00  25.13  25.170  25.13  25.15   651942